In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!nvidia-smi

Thu Apr 18 12:17:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0
!pip3 install -q -U Cython
import os
os._exit(0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
model_ids = [
    ("google/gemma-2b-it", "gemma"),
    ("google/gemma-7b-it", "gemma")
    #("mistralai/Mistral-7B-Instruct-v0.2", "mistral")
]
templates = {
    "gemma": """<start_of_turn>user
{query} <end_of_turn>\n""",
    "llama":"",
    "mistral":"""
<s>
[INST]
{query}
[/INST]
</s>

"""
}
response_template = {
    "gemma": "\n<start_of_turn>model{query}\n<end_of_turn>",
    "llama":"<s>model\n{query}\n</s>",
    "mistral":"<s>model\n{query}\n</s>"
}


In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
#model_id = model_ids[0][0]
#simple_name = model_ids[0][1]
#new_model = "gemma2b-it-tripletdolly-rebel" #Name of the model you will be pushing to huggingface model hub
#
#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_quent_type="nf4",
#    bnb_4bit_compute_dtype=torch.bfloat16
#)
#
#model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
#tokenizer = AutoTokenizer.from_pretrained(model_id,
#                                          add_eos_token=True,
#                                          padding_side="left"
#                                          )

In [ ]:
def get_completion(prompt: str, model, tokenizer) -> str:

  device = "cuda:0"
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

def get_completion_batch(prompt: list[str], model, tokenizer) -> list[str]:
  device = "cuda:0"
  encodeds = tokenizer(prompt,\
                      return_tensors="pt", add_special_tokens=True,\
                      padding=True)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=600, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded

def context_to_string(contexts):
  contexts = [f"The relation {context['type']} from {context['head']} is {context['tail']}." for context in contexts]
  return " ".join(contexts)

def get_full_input_query(input, contexts):
  return f"""
{input}
Extra data:
{context_to_string(contexts)}
"""

def generate_prompt(query: str, simpleModelName : str, appendOutput : bool, out: str | None = None):
  prefix_text = 'Below is an instruction or question that describes a task or requires an answer.'\
  'Write a response that appropriately completes the request. \n\n'

  # Samples with additional context into.
  if simpleModelName == "gemma":
    text = templates["gemma"]
    end = response_template["gemma"]
  elif simpleModelName == "mistral":
    text = templates["mistral"]
    end = response_template["mistral"]
  else:
    text = templates["mistral"]
    end = response_template["mistral"]

  query = prefix_text + query
  text = text.format(query=query)

  if appendOutput:
    assert out != None
    text = text + end.format(query=out)

  return text

def generate_prompt_batch(queries: list[str], simpleModelName : str, appendOutput : bool = False, expected_responses: list[str] | None = None):
  if expected_responses:
    assert len(queries) == len(expected_responses, "length of input and output lists do not match")
    return [generate_prompt(q, simpleModelName, appendOutput, o) for (q,o) in zip(queries, expected_responses)]
  return [generate_prompt(q, simpleModelName, appendOutput=False) for q in queries]

def get_max_new_tokens(dataset, tokenizer):
  max_new_token = 0
  for response in dataset["response"]:
    encodeds = tokenizer(response, return_tensors="pt", add_special_tokens=True)
    max_new_token = max(max_new_token, encodeds["input_ids"].shape[-1])
  return max_new_token

def formatting_prompts_func(dataset):
    output_texts = []
    for datapoint in dataset:
        text = f"### User input: {datapoint['instruction']} \n### Relations: {' '.join([f'<entry><head>{rel[0]}<rel>{rel[1]}<tail>{rel[2]}' for rel in datapoint['context']])} \n### Response: {datapoint['response']}<eos>"
        output_texts.append(text)
    return output_texts

In [ ]:
#result = get_completion(generate_prompt_batch(["code the fibonacci series in python using recursion"], simple_name), model=model, tokenizer=tokenizer)
#print(result)

In [ ]:
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import transformers
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import bitsandbytes as bnb

def finetune(model_id, simple_name):

  print("Starting the finetuning of {}".format(model_id))
  # Prepare dataset
  train_set = load_dataset("rPucs/TripletDollyQA-3k-Rebel", split="train")
  test_set = load_dataset("rPucs/TripletDollyQA-3k-Rebel", split="test")
  prompts = []
  for datapoint in train_set:
    prompts.append(generate_prompt(get_full_input_query(datapoint["instruction"], datapoint["context"]), simple_name, appendOutput=True,\
                                  out=datapoint["response"]))
  train_set = train_set.add_column("prompt", prompts)
  prompts = []
  for datapoint in test_set:
    prompts.append(generate_prompt(get_full_input_query(datapoint["instruction"], datapoint["context"]), simple_name, appendOutput=True,\
                                  out=datapoint["response"]))
  test_set = test_set.add_column("prompt", prompts)

  # Setup model
  new_model = modelname.split("/")[1] + "TripletDolly-Rebel" #Name of the model you will be pushing to huggingface model hub

  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quent_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
  tokenizer = AutoTokenizer.from_pretrained(model_id,
                                            add_eos_token=True,
                                            padding_side="left"
                                            )

  # Setup train config
  model.gradient_checkpointing_enable()
  model = prepare_model_for_kbit_training(model)
  modules = find_all_linear_names(model)

  lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CASUAL_LM"
  )

  model = get_peft_model(model, lora_config)
  tokenizer.pad_token = tokenizer.eos_token
  torch.cuda.empty_cache()

  resp_template = response_template[simple_name].split("{query}")[0]
  instruct_template = templates[simple_name].split("{query}")[0]
  print(f"Response template: {resp_template}")
  print(f"Instruction template: {instruct_template}")

  trainer = SFTTrainer(
      model=model,
      train_dataset=train_set,
      eval_dataset=test_set,
      dataset_text_field="prompt",
      peft_config=lora_config,
      args=transformers.TrainingArguments(
          per_device_train_batch_size=1,
          gradient_accumulation_steps=4,
          warmup_steps=0.03,
          max_steps=100,
          learning_rate=2e-4,
          logging_steps=1,
          output_dir="outputs",
          optim="paged_adamw_8bit",
          save_strategy="epoch",
      ),
      data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
      #data_collator=DataCollatorForCompletionOnlyLM(response_template=resp_template, instruction_template=instruct_template, tokenizer=tokenizer),
      #max_seq_length=10_000,
      packing=False
  )

  # Train
  model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
  trainer.train()
  model.config.use_cache = True

  # Upload
  print("Uploading {} to hub".format(new_model))
  trainer.model.save_pretrained(new_model)
  base_model = AutoModelForCausalLM.from_pretrained(
      model_id,
      low_cpu_mem_usage=True,
      return_dict=True,
      torch_dtype=torch.float16,
      device_map={"": 0},
  )
  merged_model= PeftModel.from_pretrained(base_model, new_model)
  merged_model= merged_model.merge_and_unload()

  # Save the merged model
  merged_model.save_pretrained("merged_model",safe_serialization=True)
  tokenizer.save_pretrained("merged_model")
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"
  # Push the model and tokenizer to the Hugging Face Model Hub
  merged_model.push_to_hub(new_model, use_temp_dir=False)
  tokenizer.push_to_hub(new_model, use_temp_dir=False)

  # Release memory
  del model, merged_model, tokenizer, base_model, trainer
  torch.cuda.empty_cache()


def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
for modelname, simple_name in [model_ids[2]]:
  finetune(modelname, simple_name)

Starting the finetuning of mistralai/Mistral-7B-Instruct-v0.2


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

In [ ]:
# import transformers

# tokenizer.pad_token = tokenizer.eos_token


# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=train_data,
#     eval_dataset=test_data,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         warmup_steps=0.03,
#         max_steps=100,
#         learning_rate=2e-4,
#         fp16=True,
#         logging_steps=1,
#         output_dir="outputs_mistral_b_finance_finetuned_test",
#         optim="paged_adamw_8bit",
#         save_strategy="epoch",
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )


Sample fine tuned model

In [ ]:
#result = get_completion(query="code the fibonacci series in python using reccursion", model=merged_model, tokenizer=tokenizer)
#print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction or question that describes a task or requires an answer. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion 

  ```python
def fibonacci(n):
  """
  Returns the nth number in the Fibonacci sequence.

  Args:
    n: The index in the sequence to return.

  Returns:
    The nth number in the Fibonacci sequence.
  """
  if n == 0:
    return 0
  elif n == 1:
    return 1
  return fibonacci(n-1) + fibonacci(n-2)
```
 risolvendo questa richiesta con la ricorsione si può trovare il programma in seguenti link
1.  Python program to print the first 20 numbers in the Fibonacci sequence
2. Python program to print first 20 numbers in the fibonacci series using recursion
3. Understanding the mechanics of recursion in Python
4. Implementing a recursive Python function to calculate the nth number in the Fibonacci sequence
5. How to use recursion to compute the n-th number in the fibonacci sequence

Grazie p

In [ ]:
!zip -r /content/outputs_gemma7_mistral02_it.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/checkpoint-100/ (stored 0%)
  adding: content/outputs/checkpoint-100/README.md (deflated 66%)
  adding: content/outputs/checkpoint-100/tokenizer.model (deflated 55%)
  adding: content/outputs/checkpoint-100/trainer_state.json (deflated 80%)
  adding: content/outputs/checkpoint-100/training_args.bin (deflated 51%)
  adding: content/outputs/checkpoint-100/rng_state.pth (deflated 25%)
  adding: content/outputs/checkpoint-100/special_tokens_map.json (deflated 73%)
  adding: content/outputs/checkpoint-100/adapter_model.safetensors (deflated 7%)
  adding: content/outputs/checkpoint-100/adapter_config.json (deflated 52%)
  adding: content/outputs/checkpoint-100/optimizer.pt (deflated 17%)
  adding: content/outputs/checkpoint-100/scheduler.pt (deflated 57%)
  adding: content/outputs/checkpoint-100/tokenizer_config.json (deflated 64%)
  adding: content/outputs/checkpoint-100/tokenizer.json (deflated 74%)
  adding: content/outputs/

In [ ]:
from google.colab import files
files.download("outputs_gemma7_mistral02_it.zip")